In [39]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory


from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [40]:
mx.context.num_gpus()

0

In [41]:
df = pd.read_csv('../../../data/Merged-update_hourly.csv')
# df = pd.read_csv('pollution.csv')

In [42]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,2.90,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0


In [43]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'Time'], infer_datetime_format=True)

In [44]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,...,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0,2010-01-01 00:00:00
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,...,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0,2010-01-01 01:00:00
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,...,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0,2010-01-01 02:00:00
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,...,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0,2010-01-01 03:00:00
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,...,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0,2020-12-31 22:00:00


In [45]:
df.columns

Index(['Time', 'WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN', 'timestamp'],
      dtype='object')

In [46]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [47]:
train_df.shape

(77145, 21)

In [48]:
test_df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
77145,10/20/18 09:00,1.06,1.01,0.0,0.0,2.41,1.07,0.0,0.0,0.0,...,1.15,0.0,469.19,0.0,2.61,2.35,1.11,0.0,0.0,2018-10-20 09:00:00
77146,10/20/18 10:00,0.68,0.63,0.0,0.0,2.41,0.71,0.0,0.0,0.0,...,0.82,0.0,0.00,0.0,0.00,2.62,0.72,0.0,0.0,2018-10-20 10:00:00
77147,10/20/18 11:00,0.31,0.32,0.0,0.0,2.41,0.33,0.0,0.0,0.0,...,0.42,0.0,0.00,0.0,0.00,2.62,0.37,0.0,0.0,2018-10-20 11:00:00
77148,10/20/18 12:00,0.20,0.23,0.0,0.0,2.40,0.24,0.0,0.0,0.0,...,0.35,0.0,0.00,0.0,0.00,2.67,0.28,0.0,0.0,2018-10-20 12:00:00
77149,10/20/18 13:00,0.41,0.42,0.0,0.0,2.39,0.42,0.0,0.0,0.0,...,0.48,0.0,0.00,0.0,0.00,2.72,0.45,0.0,0.0,2018-10-20 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.29,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.66,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.04,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.18,0.0,0.0,2020-12-31 22:00:00


In [49]:
train_ds = PandasDataset(
    train_df,
    target = ['TWS_S25B'],
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25A', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

In [50]:
test_ds = PandasDataset(
    test_df,
    target = ['TWS_S25B'],
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25A', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

#### training

In [51]:
K = 48
n_hours = 24*3

In [52]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=32,
    num_heads=2,
)

model_25b = estimator.train(train_ds)

100%|███████████████████████████████████████████████| 100/100 [05:18<00:00,  3.19s/it, epoch=130/150, avg_epoch_loss=0.0311]


100%|███████████████████████████████████████████████| 100/100 [05:28<00:00,  3.29s/it, epoch=150/150, avg_epoch_loss=0.0308]


#### predicted y

In [53]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [54]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'TWS_S25B',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25A', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model_25b, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[2.57358789 2.07051492 1.54199612 1.0633713  0.6317358  0.41008544
 0.53632367 1.04697144 1.6310755  2.11374211 2.51946664 2.72034669
 2.55947542 2.0596683  1.53807127 1.06535327 0.53599691 0.18832266
 0.22876485 0.66528875 1.33022785 1.91622543 2.41898131 2.77891421
 2.85086751 2.53864479 1.97163868 1.4437207  0.94719028 0.5103727
 0.37965569 0.63795376 1.2060672  1.75589836 2.23889947 2.62177634
 2.72663307 2.42983246 1.87414014 1.39972651 0.89357013 0.37496021
 0.11665275 0.27685586 0.81270772 1.47202718 2.03942275 2.52118659]
(48,)
------
1
[2.57358789 2.07051492 1.54199612 1.0633713  0.6317358  0.41008544
 0.53632367 1.04697144 1.6310755  2.11374211 2.51946664 2.72034669
 2.55947542 2.0596683  1.53807127 1.06535327 0.53599691 0.18832266
 0.22876485 0.66528875 1.33022785 1.91622543 2.41898131 2.77891421
 2.85086751 2.53864479 1.97163868 1.4437207  0.94719028 0.5103727
 0.37965569 0.63795376 1.2060672  1.75589836 2.23889947 2.62177634
 2.72663307 2.42983246 1.87414014 1.39972651 0

[2.57358789 2.07051492 1.54199612 1.0633713  0.6317358  0.41008544
 0.53632367 1.04697144 1.6310755  2.11374211 2.51946664 2.72034669
 2.55947542 2.0596683  1.53807127 1.06535327 0.53599691 0.18832266
 0.22876485 0.66528875 1.33022785 1.91622543 2.41898131 2.77891421
 2.85086751 2.53864479 1.97163868 1.4437207  0.94719028 0.5103727
 0.37965569 0.63795376 1.2060672  1.75589836 2.23889947 2.62177634
 2.72663307 2.42983246 1.87414014 1.39972651 0.89357013 0.37496021
 0.11665275 0.27685586 0.81270772 1.47202718 2.03942275 2.52118659
 2.06137919 1.54761791 1.07426536 0.64585799 0.41503671 0.53274876
 1.04463899 1.63359833 2.11569476 2.51784348 2.72496796 2.57899785
 2.09171581 1.57174969 1.08203876 0.54379779 0.18804626 0.22478788
 0.66041416 1.33020103 1.9184624  2.41914821 2.78087997 2.86502004
 2.5688355  2.00333238 1.46654499 0.96038866 0.51445299 0.37988415
 0.64127499 1.21380508 1.76389134 2.24284768 2.62229037 2.73525977
 2.45615482 1.90952909 1.42406213 0.91116899 0.3867971  0.12394

[2.57358789 2.07051492 1.54199612 1.0633713  0.6317358  0.41008544
 0.53632367 1.04697144 1.6310755  2.11374211 2.51946664 2.72034669
 2.55947542 2.0596683  1.53807127 1.06535327 0.53599691 0.18832266
 0.22876485 0.66528875 1.33022785 1.91622543 2.41898131 2.77891421
 2.85086751 2.53864479 1.97163868 1.4437207  0.94719028 0.5103727
 0.37965569 0.63795376 1.2060672  1.75589836 2.23889947 2.62177634
 2.72663307 2.42983246 1.87414014 1.39972651 0.89357013 0.37496021
 0.11665275 0.27685586 0.81270772 1.47202718 2.03942275 2.52118659
 2.06137919 1.54761791 1.07426536 0.64585799 0.41503671 0.53274876
 1.04463899 1.63359833 2.11569476 2.51784348 2.72496796 2.57899785
 2.09171581 1.57174969 1.08203876 0.54379779 0.18804626 0.22478788
 0.66041416 1.33020103 1.9184624  2.41914821 2.78087997 2.86502004
 2.5688355  2.00333238 1.46654499 0.96038866 0.51445299 0.37988415
 0.64127499 1.21380508 1.76389134 2.24284768 2.62229037 2.73525977
 2.45615482 1.90952909 1.42406213 0.91116899 0.3867971  0.12394

[2.57358789 2.07051492 1.54199612 1.0633713  0.6317358  0.41008544
 0.53632367 1.04697144 1.6310755  2.11374211 2.51946664 2.72034669
 2.55947542 2.0596683  1.53807127 1.06535327 0.53599691 0.18832266
 0.22876485 0.66528875 1.33022785 1.91622543 2.41898131 2.77891421
 2.85086751 2.53864479 1.97163868 1.4437207  0.94719028 0.5103727
 0.37965569 0.63795376 1.2060672  1.75589836 2.23889947 2.62177634
 2.72663307 2.42983246 1.87414014 1.39972651 0.89357013 0.37496021
 0.11665275 0.27685586 0.81270772 1.47202718 2.03942275 2.52118659
 2.06137919 1.54761791 1.07426536 0.64585799 0.41503671 0.53274876
 1.04463899 1.63359833 2.11569476 2.51784348 2.72496796 2.57899785
 2.09171581 1.57174969 1.08203876 0.54379779 0.18804626 0.22478788
 0.66041416 1.33020103 1.9184624  2.41914821 2.78087997 2.86502004
 2.5688355  2.00333238 1.46654499 0.96038866 0.51445299 0.37988415
 0.64127499 1.21380508 1.76389134 2.24284768 2.62229037 2.73525977
 2.45615482 1.90952909 1.42406213 0.91116899 0.3867971  0.12394

[2.57358789 2.07051492 1.54199612 ... 0.42532539 1.00333393 1.63076127]
(1152,)
------
24
[2.57358789 2.07051492 1.54199612 ... 0.96829748 1.60481131 2.15660024]
(1200,)
------
25
[2.57358789 2.07051492 1.54199612 ... 1.60251772 2.1496799  2.59467578]
(1248,)
------
26
[2.57358789 2.07051492 1.54199612 ... 2.15898728 2.60173869 2.81826019]
(1296,)
------
27
[2.57358789 2.07051492 1.54199612 ... 2.60549116 2.81430936 2.64856887]
(1344,)
------
28
[2.57358789 2.07051492 1.54199612 ... 2.80338931 2.63037038 2.11357856]
(1392,)
------
29
[2.57358789 2.07051492 1.54199612 ... 2.59947062 2.08470035 1.54513276]
(1440,)
------
30
[2.57358789 2.07051492 1.54199612 ... 2.04500079 1.5031873  1.01933312]
(1488,)
------
31
[2.57358789 2.07051492 1.54199612 ... 1.47738135 0.98824555 0.52133292]
(1536,)
------
32
[2.57358789 2.07051492 1.54199612 ... 0.92920476 0.46431962 0.24767749]
(1584,)
------
33
[2.57358789 2.07051492 1.54199612 ... 0.47246099 0.25323081 0.40041646]
(1632,)
------
34
[2.5735878

...

[2.57358789 2.07051492 1.54199612 ... 1.35533047 2.08932972 2.73587918]
(392304,)
------
8173
[2.57358789 2.07051492 1.54199612 ... 2.08455849 2.72634196 3.26168633]
(392352,)
------
8174
[2.57358789 2.07051492 1.54199612 ... 2.72901082 3.26090455 3.5051384 ]
(392400,)
------
8175
[2.57358789 2.07051492 1.54199612 ... 3.26022601 3.50383067 3.26580501]
(392448,)
------
8176
[2.57358789 2.07051492 1.54199612 ... 3.4939363  3.24126458 2.60622907]
(392496,)
------
8177
[2.57358789 2.07051492 1.54199612 ... 3.24260736 2.61576056 1.93891573]
(392544,)
------
8178
[2.57358789 2.07051492 1.54199612 ... 2.63218284 1.95176888 1.32763815]
(392592,)
------
8179
[2.57358789 2.07051492 1.54199612 ... 1.98484504 1.35430562 0.68488616]
(392640,)
------
8180
[2.57358789 2.07051492 1.54199612 ... 1.41940641 0.75453472 0.31202671]
(392688,)
------
8181
[2.57358789 2.07051492 1.54199612 ... 0.81668955 0.38097903 0.43632638]
(392736,)
------
8182
[2.57358789 2.07051492 1.54199612 ... 0.35639322 0.40035677 

[2.57358789 2.07051492 1.54199612 ... 1.96445382 2.59793663 3.12642193]
(396528,)
------
8261
[2.57358789 2.07051492 1.54199612 ... 2.60895228 3.13870883 3.45975423]
(396576,)
------
8262
[2.57358789 2.07051492 1.54199612 ... 3.13755345 3.46162915 3.41736341]
(396624,)
------
8263
[2.57358789 2.07051492 1.54199612 ... 3.44906616 3.43754005 3.02968049]
(396672,)
------
8264
[2.57358789 2.07051492 1.54199612 ... 3.41028976 3.02351856 2.48311448]
(396720,)
------
8265
[2.57358789 2.07051492 1.54199612 ... 3.02129841 2.48961949 2.02369642]
(396768,)
------
8266
[2.57358789 2.07051492 1.54199612 ... 2.46157455 1.99962127 1.55902016]
(396816,)
------
8267
[2.57358789 2.07051492 1.54199612 ... 2.01537085 1.56983602 1.24178112]
(396864,)
------
8268
[2.57358789 2.07051492 1.54199612 ... 1.6368345  1.30367208 1.27006161]
(396912,)
------
8269
[2.57358789 2.07051492 1.54199612 ... 1.27485573 1.2464875  1.5451951 ]
(396960,)
------
8270
[2.57358789 2.07051492 1.54199612 ... 1.20361888 1.48736727 

[2.57358789 2.07051492 1.54199612 ... 2.60245132 2.88883281 3.00262761]
(400800,)
------
8350
[2.57358789 2.07051492 1.54199612 ... 2.89063001 3.00040364 2.84210896]
(400848,)
------
8351
[2.57358789 2.07051492 1.54199612 ... 3.0088644  2.84887028 2.4549768 ]
(400896,)
------
8352
[2.57358789 2.07051492 1.54199612 ... 2.83001304 2.43258214 2.00835299]
(400944,)
------
8353
[2.57358789 2.07051492 1.54199612 ... 2.44159579 2.01400256 1.61450911]
(400992,)
------
8354
[2.57358789 2.07051492 1.54199612 ... 1.98242664 1.57114935 1.17973757]
(401040,)
------
8355
[2.57358789 2.07051492 1.54199612 ... 1.58614743 1.1968044  0.98752165]
(401088,)
------
8356
[2.57358789 2.07051492 1.54199612 ... 1.20226896 0.99059439 1.07920253]
(401136,)
------
8357
[2.57358789 2.07051492 1.54199612 ... 0.95749551 1.04848731 1.43649673]
(401184,)
------
8358
[2.57358789 2.07051492 1.54199612 ... 1.01052046 1.39667499 1.89971352]
(401232,)
------
8359
[2.57358789 2.07051492 1.54199612 ... 1.38520002 1.88891351 

[2.57358789 2.07051492 1.54199612 ... 2.85202956 2.42858171 2.05263948]
(405120,)
------
8440
[2.57358789 2.07051492 1.54199612 ... 2.42424798 2.04679656 1.69301975]
(405168,)
------
8441
[2.57358789 2.07051492 1.54199612 ... 2.0448904  1.68684578 1.29971611]
(405216,)
------
8442
[2.57358789 2.07051492 1.54199612 ... 1.67124987 1.27413332 1.05835247]
(405264,)
------
8443
[2.57358789 2.07051492 1.54199612 ... 1.27470315 1.06015825 1.14796317]
(405312,)
------
8444
[2.57358789 2.07051492 1.54199612 ... 1.14431632 1.21433938 1.55970669]
(405360,)
------
8445
[2.57358789 2.07051492 1.54199612 ... 1.24775064 1.58453763 2.05946398]
(405408,)
------
8446
[2.57358789 2.07051492 1.54199612 ... 1.57295096 2.053828   2.48980117]
(405456,)
------
8447
[2.57358789 2.07051492 1.54199612 ... 2.08594847 2.52830315 2.92252851]
(405504,)
------
8448
[2.57358789 2.07051492 1.54199612 ... 2.53055501 2.90904236 3.1579926 ]
(405552,)
------
8449
[2.57358789 2.07051492 1.54199612 ... 2.90921807 3.14352345 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [55]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(19168, 48)

In [56]:
test_ds_predict_q50_supervised[0:3]

array([[2.57358789, 2.07051492, 1.54199612, 1.0633713 , 0.6317358 ,
        0.41008544, 0.53632367, 1.04697144, 1.6310755 , 2.11374211,
        2.51946664, 2.72034669, 2.55947542, 2.0596683 , 1.53807127,
        1.06535327, 0.53599691, 0.18832266, 0.22876485, 0.66528875,
        1.33022785, 1.91622543, 2.41898131, 2.77891421, 2.85086751,
        2.53864479, 1.97163868, 1.4437207 , 0.94719028, 0.5103727 ,
        0.37965569, 0.63795376, 1.2060672 , 1.75589836, 2.23889947,
        2.62177634, 2.72663307, 2.42983246, 1.87414014, 1.39972651,
        0.89357013, 0.37496021, 0.11665275, 0.27685586, 0.81270772,
        1.47202718, 2.03942275, 2.52118659],
       [2.06137919, 1.54761791, 1.07426536, 0.64585799, 0.41503671,
        0.53274876, 1.04463899, 1.63359833, 2.11569476, 2.51784348,
        2.72496796, 2.57899785, 2.09171581, 1.57174969, 1.08203876,
        0.54379779, 0.18804626, 0.22478788, 0.66041416, 1.33020103,
        1.9184624 , 2.41914821, 2.78087997, 2.86502004, 2.5688355 ,
   

#### actual y

In [57]:
from helper import series_to_supervised, stage_series_to_supervised

In [58]:
test_df_supervised = series_to_supervised(test_df[['WS_S1']], n_hours, K)

In [59]:
test_df_supervised.iloc[:, -K:].shape

(19168, 48)

In [60]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [61]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+38),var1(t+39),var1(t+40),var1(t+41),var1(t+42),var1(t+43),var1(t+44),var1(t+45),var1(t+46),var1(t+47)
77217,2.53,1.92,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,...,1.62,1.06,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48
77218,1.92,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,...,1.06,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80
77219,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,...,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70
77220,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,2.59,...,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70,2.19
77221,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,2.59,2.05,...,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70,2.19,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96380,1.95,1.96,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,...,2.36,2.18,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24
96381,1.96,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,...,2.18,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62
96382,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,1.59,...,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62,1.98
96383,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,1.59,1.99,...,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62,1.98,2.13


#### performance on test set

In [62]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 0.158
RMSE = 0.21
